In [17]:
from flask import Flask, request, render_template_string, redirect, url_for, render_template
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
import base64

db_creds = pd.read_csv('/users/zack burnside/desktop/database_creds.csv')

app = Flask(__name__)

def fetch_user_results(username=None):
    conn = psycopg2.connect(
        dbname=db_creds['dbname'][0],
        user=db_creds['user'][0],
        password=db_creds['password'][0],
        host=db_creds['host'][0],
        port=db_creds['port'][0]
    )
    cur = conn.cursor()
    if username:
        cur.execute("SELECT user_name, player_pick, prize_money, tournament_name FROM public.weekly_tournament_results WHERE user_name = %s", (username,))
    else:
        cur.execute("SELECT user_name, player_pick, prize_money, tournament_name FROM public.weekly_tournament_results")
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

def fetch_tournament_results(tournament_name):
    conn = psycopg2.connect(
        dbname=db_creds['dbname'][0],
        user=db_creds['user'][0],
        password=db_creds['password'][0],
        host=db_creds['host'][0],
        port=db_creds['port'][0]
    )
    cur = conn.cursor()
    cur.execute("""
        SELECT user_name, player_pick, prize_money 
        FROM public.weekly_tournament_results 
        WHERE tournament_name = %s
    """, (tournament_name,))
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

def fetch_leaderboard():
    conn = psycopg2.connect(
        dbname=db_creds['dbname'][0],
        user=db_creds['user'][0],
        password=db_creds['password'][0],
        host=db_creds['host'][0],
        port=db_creds['port'][0]
    )
    cur = conn.cursor()
    cur.execute("""
        SELECT user_name, SUM(prize_money) as total_prize_money
        FROM public.weekly_tournament_results
        GROUP BY user_name
        ORDER BY total_prize_money DESC
    """)
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

def insert_pick(username, tournament, pick):
    conn = psycopg2.connect(
        dbname=db_creds['dbname'][0],
        user=db_creds['user'][0],
        password=db_creds['password'][0],
        host=db_creds['host'][0],
        port=db_creds['port'][0]
    )
    cur = conn.cursor()
    cur.execute("""
        INSERT INTO public.weekly_tournament_results (user_name, tournament_name, player_pick)
        VALUES (%s, %s, %s)
    """, (username, tournament, pick))
    conn.commit()
    cur.close()
    conn.close()

@app.route('/results', methods=['GET', 'POST'])
def results():
    selected_user = None
    if request.method == 'POST':
        selected_user = request.form['username']
    
    results_data = fetch_user_results(selected_user)
    usernames = [row[0] for row in fetch_user_results()]  # Fetch usernames for the dropdown
    
    return render_template_string('''
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f4f4f4;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                flex-direction: column;
            }
            table {
                width: 80%;
                margin-top: 20px;
                border-collapse: collapse;
            }
            table, th, td {
                border: 1px solid #ccc;
            }
            th, td {
                padding: 10px;
                text-align: left;
            }
            th {
                background-color: #4CAF50;
                color: white;
            }
            .back-button {
                margin-top: 20px;
                display: inline-block;
                background-color: #4CAF50;
                color: white;
                padding: 10px 15px;
                text-decoration: none;
                border-radius: 4px;
            }
            .filter-form {
                margin-top: 20px;
                margin-bottom: 20px;
            }
            .filter-form select {
                padding: 10px;
                margin-right: 10px;
            }
            .filter-form input[type="submit"] {
                padding: 10px 15px;
                background-color: #4CAF50;
                color: white;
                border: none;
                border-radius: 4px;
                cursor: pointer;
            }
        </style>
        <h1>Results</h1>
        <form method="post" class="filter-form">
            <select name="username">
                <option value="">All Users</option>
                {% for name in usernames %}
                    <option value="{{ name }}" {% if selected_user == name %}selected{% endif %}>{{ name }}</option>
                {% endfor %}
            </select>
            <input type="submit" value="Filter">
        </form>
        <table>
            <thead>
                <tr>
                    <th>User Name</th>
                    <th>Player Pick</th>
                    <th>Prize Money</th>
                    <th>Tournament Name</th>
                </tr>
            </thead>
            <tbody>
                {% for user_name, player_pick, prize_money, tournament_name in results_data %}
                    <tr>
                        <td>{{ user_name }}</td>
                        <td>{{ player_pick }}</td>
                        <td>${{ prize_money }}</td>
                        <td>{{ tournament_name }}</td>
                    </tr>
                {% endfor %}
            </tbody>
        </table>
        <a href="/" class="back-button">Back to Form</a>
    ''', results_data=results_data, usernames=set(usernames), selected_user=selected_user)

@app.route('/', methods=['GET', 'POST'])
def main_page():
    if request.method == 'POST':
        username = request.form['username']
        tournament = request.form['tournament']
        pick = request.form['pick']
        
        # Insert the pick into the database
        # Assuming you have a function insert_pick defined elsewhere
        insert_pick(username, tournament, pick)
        
        return f'Thank you! Your pick is {pick}.'
    
    usernames = [
        "Z", "Timmy Chips", "Goob", "C", "Steve", "Sus", "DV3", "Jlo", 
        "Ty Dolla $$$", "Ryan", "Mike D", "Mando", "Eddie Buckets", 
        "Garrett", "T Sully"
    ]
    
    tournaments = [
        "WM Phoenix Open", "The Genesis Invitational", "Mexico Open at Vidanta", 
        "Cognizant Classic", "Arnold Palmer Invitational", 
        "THE PLAYERS Championship", "Valspar Championship", "Texas Children's Houston Open", 
        "The Valero Texas Open", "Masters Tournament", "RBC Heritage", "THE CJ CUP Byron Nelson", 
        "Wells Fargo Championship", "PGA Championship", "Charles Schwab Challenge", 
        "RBC Canadian Open", "the Memorial Tournament presented by Workday", "U.S. Open", 
        "Travelers Championship", "Rocket Mortgage Classic", "John Deere Classic", 
        "Genesis Scottish Open", "The Open Championship", "3M Open", "Wyndham Championship", 
        "FedEx St. Jude Championship", "BMW Championship", "TOUR Championship"
    ]
    
    return render_template_string('''
        <form method="post">
            Username: 
            <select name="username">
                {% for name in usernames %}
                    <option value="{{ name }}">{{ name }}</option>
                {% endfor %}
            </select><br>
            Tournament: 
            <select name="tournament">
                {% for tournament in tournaments %}
                    <option value="{{ tournament }}">{{ tournament }}</option>
                {% endfor %}
            </select><br>
            Golfer Pick: <input type="text" name="pick"><br>
            <input type="submit" value="Submit">
        </form>
        <br>
        <a href="/leaderboard">View Leaderboard</a>
        <br>
        <h2>Past Tournament Results</h2>
        {% for tournament in tournaments %}
            <a href="/tournament_results/{{ tournament }}">{{ tournament }}</a><br>
        {% endfor %}
        <br>
        <a href="/stats">View Stats</a>
    ''', usernames=usernames, tournaments=tournaments)

def fetch_stats_data():
    conn = psycopg2.connect(
        dbname=db_creds['dbname'][0],
        user=db_creds['user'][0],
        password=db_creds['password'][0],
        host=db_creds['host'][0],
        port=db_creds['port'][0]
    )
    cursor = conn.cursor()
    cursor.execute("SELECT player_pick, COUNT(*) as pick_count FROM public.user_picks GROUP BY player_pick ORDER BY pick_count DESC")
    data = cursor.fetchall()
    conn.close()
    return data

def create_bar_chart(data):
    golfers = [row[0] for row in data]
    pick_count = [row[1] for row in data]

    fig, ax = plt.subplots(figsize=(20, 12))
    bars = ax.bar(golfers, pick_count, color='skyblue')

    for bar in bars:
        count_value = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), str(count_value), ha='center', va='bottom', fontsize=8)

    ax.set_title('Most Picked Golfer', fontsize=12)
    ax.set_xlabel('Golfer Name', fontsize=10)
    ax.set_ylabel('Pick Count', fontsize=10)
    plt.xticks(rotation=45, ha='right', fontsize=8)
    plt.grid(axis='y', linestyle='--')
    plt.tight_layout()

    # Convert plot to PNG image and encode it in base64
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)
    plot_data = buffer.getvalue()
    buffer.close()
    plot_data = base64.b64encode(plot_data).decode('utf-8')
    plt.close()

    return plot_data
# Function to create a bar chart using Matplotlib

@app.route('/stats')
def stats_page():
    # Fetch data and create chart (example)
    stats_data = fetch_stats_data()  # Function to fetch your data
    plot_data = create_bar_chart(stats_data)  # Function to create your chart

    # Convert plot_data to base64-encoded image (example)

    # HTML content directly in the route function
    html_content = f'''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Statistics Page</title>
    </head>
    <body>
        <h1>Most Picked Golfer</h1>

        <!-- Display the chart using base64-encoded image -->
        <img src="data:image/png;base64, {plot_data}" alt="Chart">

        <!-- Add other HTML elements as needed -->

    </body>
    </html>
    '''

    return render_template_string(html_content)

@app.route('/leaderboard')
def leaderboard():
    leaderboard_data = fetch_leaderboard()
    return render_template_string('''
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f4f4f4;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                flex-direction: column;
            }
            table {
                width: 80%;
                margin-top: 20px;
                border-collapse: collapse;
            }
            table, th, td {
                border: 1px solid #ccc;
            }
            th, td {
                padding: 10px;
                text-align: left;
            }
            th {
                background-color: #4CAF50;
                color: white;
            }
            .back-button {
                margin-top: 20px;
                display: inline-block;
                background-color: #4CAF50;
                color: white;
                padding: 10px 15px;
                text-decoration: none;
                border-radius: 4px;
            }
        </style>
        <h1>Leaderboard</h1>
        <table>
            <thead>
                <tr>
                    <th>User Name</th>
                    <th>Total Prize Money</th>
                </tr>
            </thead>
            <tbody>
                {% for user_name, total_prize_money in leaderboard_data %}
                    <tr>
                        <td>{{ user_name }}</td>
                        <td>{{ total_prize_money }}</td>
                    </tr>
                {% endfor %}
            </tbody>
        </table>
        <a href="/" class="back-button">Back to Form</a>
    ''', leaderboard_data=leaderboard_data)

@app.route('/tournament_results/<tournament_name>')
def tournament_results(tournament_name):
    results_data = fetch_tournament_results(tournament_name)
    return render_template_string('''
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f0f0f0;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                flex-direction: column;
                margin: 0;
                padding: 20px;
            }
            .container {
                width: 80%;
                background-color: #fff;
                padding: 20px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                border-radius: 8px;
            }
            h1 {
                margin-bottom: 20px;
                font-size: 24px;
                color: #333;
            }
            table {
                width: 100%;
                border-collapse: collapse;
                margin-bottom: 20px;
            }
            table, th, td {
                border: 1px solid #ddd;
            }
            th, td {
                padding: 12px;
                text-align: left;
            }
            th {
                background-color: #4CAF50;
                color: white;
            }
            tr:nth-child(even) {
                background-color: #f9f9f9;
            }
            tr:hover {
                background-color: #f1f1f1;
            }
            .back-button, .tournament-button {
                display: inline-block;
                background-color: #4CAF50;
                color: white;
                padding: 10px 15px;
                text-decoration: none;
                border-radius: 4px;
                transition: background-color 0.3s ease;
                margin: 5px;
            }
            .back-button:hover, .tournament-button:hover {
                background-color: #45a049;
            }
            .tournament-buttons {
                margin-top: 20px;
                display: flex;
                flex-wrap: wrap;
                justify-content: center;
            }
            .tournament-button {
                margin: 5px;
            }
        </style>
        <div class="container">
            <h1>Tournament Results: {{ tournament_name }}</h1>
            <table>
                <thead>
                    <tr>
                        <th>User Name</th>
                        <th>Player Pick</th>
                        <th>Prize Money</th>
                    </tr>
                </thead>
                <tbody>
                    {% for user_name, player_pick, prize_money in results_data %}
                        <tr>
                            <td>{{ user_name }}</td>
                            <td>{{ player_pick }}</td>
                            <td>{{ prize_money }}</td>
                        </tr>
                    {% endfor %}
                </tbody>
            </table>
            <a href="/" class="back-button">Back to Form</a>
            <div class="tournament-buttons">
                {% for tournament in tournaments %}
                    <a href="/tournament_results/{{ tournament }}" class="tournament-button">{{ tournament }}</a>
                {% endfor %}
            </div>
        </div>
    ''', tournament_name=tournament_name, results_data=results_data, tournaments=[
        "WM Phoenix Open", "The Genesis Invitational", "Mexico Open at Vidanta", 
        "Cognizant Classic", "Arnold Palmer Invitational", 
        "THE PLAYERS Championship", "Valspar Championship", "Texas Children's Houston Open", 
        "The Valero Texas Open", "Masters Tournament", "RBC Heritage", "THE CJ CUP Byron Nelson", 
        "Wells Fargo Championship", "PGA Championship", "Charles Schwab Challenge", 
        "RBC Canadian Open", "the Memorial Tournament presented by Workday", "U.S. Open", 
        "Travelers Championship", "Rocket Mortgage Classic", "John Deere Classic", 
        "Genesis Scottish Open", "The Open Championship", "3M Open", "Wyndham Championship", 
        "FedEx St. Jude Championship", "BMW Championship", "TOUR Championship"
    ])


if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Jul/2024 23:28:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2024 23:28:38] "GET /stats HTTP/1.1" 200 -
